# Fantasy Footbal VBD Auction Creator and Optimizer
1. Creates Value-based drafting values from 'Points above replacement' concept
2. Determines optimal split per cost on any position (eg. 80 to RB = 60 RB1; 20 RB2)
3. Runs optimal team splits (% on each position)

Remaining issues:
- QB values are wonky
    - Normalize for prior year values?
    - Manually adjust upwards?
- Optimizating of team splits has too many calculations
    - Narrow search ranges
    - Increase percentage increment?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df_raw_stats = pd.read_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2018 Data\Combined_data.csv')

In [3]:
df_raw_stats = df_raw_stats.fillna(0)
df_raw_stats = df_raw_stats.where(df_raw_stats['Player']!='Derrius Guice').dropna()
df_raw_stats['Position_Rank'] = df_raw_stats.groupby('Pos')['Exp_PTS'].rank(ascending=False)


### Define league settings

In [4]:
teams = 12
buyin = 200
qb_start, qb_bench = 2,0.35
rb_start, rb_bench = 2.5, .5
wr_start, wr_bench = 2.5, 2
te_start, te_bench = 1, .25
bench_size = 5 ### actual 6, but calculated to optimize for 2 flex. 
d_start = 1
k_start = 1
rb_waiver = int(teams*(rb_start+rb_bench)+1)
wr_waiver = int(teams*(wr_start+wr_bench)+1)
qb_waiver = int(teams*(qb_start+qb_bench)+1)
te_waiver = int(teams*(te_start+te_bench)+1)


## Run assumptions

In [5]:
starters = .94
### position multipliers based on prior years



# Replacement values

In [6]:
# Find replacement-level RB.
#### Replacement = (rb_start+rb_bench)*teams+1
#### Look up points of replacement-level by pos. 
df_rb = df_raw_stats.where(df_raw_stats['Pos']=='RB')
repl_rb=list(df_rb.where(df_rb['Position_Rank']==int(rb_waiver)).dropna().iloc[0])[15]

df_qb = df_raw_stats.where(df_raw_stats['Pos']=='QB')
repl_qb=list(df_qb.where(df_qb['Position_Rank']==int(qb_waiver)).dropna().iloc[0])[15]

df_wr = df_raw_stats.where(df_raw_stats['Pos']=='WR')
repl_wr=list(df_wr.where(df_wr['Position_Rank']==int(wr_waiver)).dropna().iloc[0])[15]

df_te = df_raw_stats.where(df_raw_stats['Pos']=='TE')
repl_te=list(df_te.where(df_te['Position_Rank']==int(te_waiver)).dropna().iloc[0])[15]

In [7]:
df_rb['PAR'] = np.max(df_rb['Exp_PTS']-0,0)
df_rb['PAR'] =df_rb.apply(lambda row: df_rb['Exp_PTS'] -repl_rb,axis=0)
df_rb.loc[df_rb['PAR']<0,'PAR'] = 0

df_wr['PAR'] = np.max(df_wr['Exp_PTS']-0,0)
df_wr['PAR'] =df_wr.apply(lambda row: df_wr['Exp_PTS'] -repl_wr,axis=0)
df_wr.loc[df_wr['PAR']<0,'PAR'] = 0

df_qb['PAR'] = np.max(df_qb['Exp_PTS']-0,0)
df_qb['PAR'] =df_qb.apply(lambda row: df_qb['Exp_PTS'] -repl_qb,axis=0)
df_qb.loc[df_qb['PAR']<0,'PAR'] = 0

df_te['PAR'] = np.max(df_te['Exp_PTS']-0,0)
df_te['PAR'] =df_te.apply(lambda row: df_te['Exp_PTS'] -repl_te,axis=0)
df_te.loc[df_te['PAR']<0,'PAR'] = 0



In [8]:
df_op = df_rb.append(df_qb, ignore_index=True)
df_op = df_op.append(df_te, ignore_index=True)
df_op = df_op.append(df_wr, ignore_index=True)
df_op.dropna()
total_par = np.sum(df_op['PAR'])
value_per_pt = teams * (buyin*starters) / total_par

In [9]:
df_op['Auction Value'] = np.max(df_op['PAR'],0)
df_op['Auction Value'] = df_op.apply(lambda row: df_op['PAR'] * value_per_pt,axis=0)
df_op['Downside'] = (df_op['Lo_PTS']-df_op['Exp_PTS'])*value_per_pt
df_op['Upside'] = (df_op['Hi_PTS']-df_op['Exp_PTS'])*value_per_pt
df_op.dropna()
df_op.sort_values(by=['Auction Value'], ascending=False)
df_pres = df_op[['Pos','Player','Team','Auction Value','Upside','Downside','Exp_PTS','Hi_PTS','Lo_PTS']]
df_pres = df_pres.sort_values(by=['Auction Value'], ascending=False)



In [10]:
df_pres.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2018 Data\Auction_values.csv')

### Sanity Check on Auction Values
- Compare values for top positions vs. prior years.

In [11]:
#prior year table:
qb_tiers_prior = [3,6,31,39,43,45]
rb_tiers_prior = [10,14,32,43,52,60]
wr_tiers_prior = [8,10,23,31,38,43]
te_tiers_prior = [0,0,6,12,17,22]

qb_tiers = []
rb_tiers = []
wr_tiers = []
te_tiers = []


tiers = [40,30,20,10,5,3]


for x in tiers:
    if x > 20:
        tiers_cap = 20
    else: 
        tiers_cap = 0    
    temp = df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='QB', df_op['Auction Value']>0)).dropna()[tiers_cap:x].mean()
    qb_tiers.append(temp)
    
for x in tiers:
    if x > 20:
        tiers_cap = 20
    else: 
        tiers_cap = 0
    temp = df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='RB', df_op['Auction Value']>0)).dropna()[tiers_cap:x].mean()
    rb_tiers.append(temp)
    
for x in tiers:
    if x > 20:
        tiers_cap = 20
    else: 
        tiers_cap = 0
    temp = df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='WR', df_op['Auction Value']>0)).dropna()[tiers_cap:x].mean()
    wr_tiers.append(temp)
for x in tiers:
    if x > 20:
        tiers_cap = 20
    else: 
        tiers_cap = 0
    temp = df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='TE', df_op['Auction Value']>0)).dropna()[tiers_cap:x].mean()
    te_tiers.append(temp)
    
    
    
df_tiers = pd.DataFrame(np.column_stack([qb_tiers,qb_tiers_prior,rb_tiers,rb_tiers_prior,wr_tiers,wr_tiers_prior,te_tiers,te_tiers_prior]),
                      columns =['qb_tiers','qb_tiers_prior','rb_tiers','rb_tiers_prior','wr_tiers','wr_tiers_prior','te_tiers','te_tiers_prior'])
df_tiers



,qb_tiers,qb_tiers_prior,rb_tiers,rb_tiers_prior,wr_tiers,wr_tiers_prior,te_tiers,te_tiers_prior
0,10.175974,3.0,8.151986,10.0,9.022996,8.0,NaN,0.0
1,10.175974,6.0,10.791015,14.0,11.131709,10.0,NaN,0.0
2,26.018217,31.0,35.005351,32.0,23.389499,23.0,9.242952,6.0
3,31.885316,39.0,47.119691,43.0,29.242253,31.0,13.312147,12.0
4,37.296962,43.0,55.177987,52.0,33.796786,38.0,19.093176,17.0
5,40.919071,45.0,59.531690,60.0,36.866613,43.0,23.788768,22.0


## Optimization function
- Test splits of spending on each position (eg. RB1/RB2) for any given amount. 
- Given these values, then run tests for % contribution to positions


### By position first...

In [12]:
def optimal_split(test_pos,budget):
    if test_pos == "Flex":
        df_temp = df_op.where(np.logical_and(df_op['Pos']!='QB', df_op['Auction Value']>0)).dropna()
    else:
        df_temp = df_op.where(np.logical_and(df_op['Pos']==test_pos, df_op['Auction Value']>0)).dropna()
    points_list = []
    cost_list = []
    
    for pl1 in range(df_temp.shape[0]):
        pl1_name = df_temp.iloc[pl1,1]
        pl1_cost = int(df_temp.iloc[pl1,20])
        pl1_pts = int(df_temp.iloc[pl1,15])
        for pl2 in range(pl1+1,df_temp.shape[0]):
            pl2_name = df_temp.iloc[pl2,1]
            pl2_cost = int(df_temp.iloc[pl2,20])
            pl2_pts = int(df_temp.iloc[pl2,15])
            total_cost = int(pl1_cost + pl2_cost)
            total_pts = int(pl1_pts + pl2_pts)
            cost_list.append(total_cost)
            points_list.append(total_pts)


    df_pairs = pd.DataFrame(np.column_stack([points_list, cost_list]), 
                           columns=['Total Pts', 'Total Cost'])
    df_pairs['Total Pts'] = pd.to_numeric(df_pairs['Total Pts'])
    df_pairs['Total Cost'] = pd.to_numeric(df_pairs['Total Cost'])

    df_max = df_pairs.where(df_pairs['Total Cost']<budget).dropna().reset_index()
    return df_max.iloc[df_max['Total Pts'].idxmax(),:]


In [13]:
def optimal_split_viewer(test_pos,budget):
    if test_pos == "Flex":
        df_temp = df_op.where(np.logical_and(df_op['Pos']!='QB', df_op['Auction Value']>0)).dropna()
    else:
        df_temp = df_op.where(np.logical_and(df_op['Pos']==test_pos, df_op['Auction Value']>0)).dropna()
    points_list = []
    cost_list = []
    pl1_list = []
    pl2_list = []
    pl1_clist = []
    pl2_clist = []
    pl1_plist = []
    pl2_plist = []
    lookup_list = []
    
    for pl1 in range(df_temp.shape[0]):
        pl1_name = df_temp.iloc[pl1,1]
        pl1_cost = int(df_temp.iloc[pl1,20])
        pl1_pts = int(df_temp.iloc[pl1,15])
        for pl2 in range(pl1+1,df_temp.shape[0]):
            pl2_name = df_temp.iloc[pl2,1]
            pl2_cost = int(df_temp.iloc[pl2,20])
            pl2_pts = int(df_temp.iloc[pl2,15])
            total_cost = int(pl1_cost + pl2_cost)
            total_pts = int(pl1_pts + pl2_pts)
            lookup = str(test_pos+str(int(budget)))
            # Create Lists
            cost_list.append(total_cost)
            points_list.append(total_pts)
            pl1_list.append(pl1_name)
            pl2_list.append(pl2_name)
            pl1_clist.append(pl1_cost)
            pl2_clist.append(pl2_cost)
            pl1_plist.append(pl1_pts)
            pl2_plist.append(pl2_pts)
            lookup_list.append(lookup)

            
    df_pairs = pd.DataFrame(np.column_stack([lookup_list,pl1_list, pl2_list, points_list, cost_list, pl1_clist, pl2_clist,pl1_plist,pl2_plist]), 
                           columns=['Lookup','Player1', 'Player2', 'Total Pts', 'Total Cost','P1 Cost','P2 Cost','P1 Pts','P2 Pts'])
    df_pairs['Total Pts'] = pd.to_numeric(df_pairs['Total Pts'])
    df_pairs['Total Cost'] = pd.to_numeric(df_pairs['Total Cost'])

    df_max = df_pairs.where(df_pairs['Total Cost']<budget).dropna().reset_index()
    return df_max.iloc[df_max['Total Pts'].idxmax(),:]


### Now for whole team... 

In [14]:
# calc min by positions
qb_adjust = 1

qb_min = int(qb_adjust*df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='QB', df_op['Auction Value']>0)).dropna()[-2:].sum()+1)
rb_min = max(int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='RB', df_op['Auction Value']>0)).dropna()[-2:].sum()+1),2)
wr_min = max(int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='WR', df_op['Auction Value']>0)).dropna()[-2:].sum()+1),2)
te_min = int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='TE', df_op['Auction Value']>0)).dropna()[-2:].sum()+1)
flex_min = max(int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']!='QB', df_op['Auction Value']>0)).dropna()[-2:].sum()+1),2)

qb_max = int(qb_adjust*df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='QB', df_op['Auction Value']>0)).dropna()[:2].sum()+2)
rb_max = max(int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='RB', df_op['Auction Value']>0)).dropna()[:2].sum()+2),2)
wr_max = max(int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='WR', df_op['Auction Value']>0)).dropna()[:2].sum()+2),2)
te_max = int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='TE', df_op['Auction Value']>0)).dropna()[:1].sum()+2)
flex_max = max(int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']!='QB', df_op['Auction Value']>0)).dropna()[:2].sum()+2),2)

print(qb_min,rb_min,wr_min,te_min,flex_min)
print(qb_max,rb_max,wr_max,te_max,flex_max)

14 2 2 1 2
87 125 78 33 125


In [15]:
df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='RB', df_op['Auction Value']>0)).dropna()[:2]

0    63.189661
1    60.643426
Name: Auction Value, dtype: float64

#### Loop throuhg possible values, append all position results to list. 
1. Only ~100 per position. 
2. Then, can be called when necessary on splits_list; do not need to rerun. 
3. df index should be cost-pos, using optimal_split_viewer


In [16]:

# we know we're gonna have 5 rows of data
#numberOfRows = (qb_max-qb_min)+(rb_max-rb_min)+(wr_max-wr_min)+(flex_max-flex_min)+(te_max-te_min)
# create dataframe
#df_pairs = pd.DataFrame(index=np.arange(0, numberOfRows), columns=('Index','Lookup','Player1', 'Player2', 'Total Pts', 'Total Cost','P1 Cost','P2 Cost','P1 Pts','P2 Pts') )

df_pairs = pd.DataFrame(columns=('Index','Lookup','Player1', 'Player2', 'Total Pts', 'Total Cost','P1 Cost','P2 Cost','P1 Pts','P2 Pts') )

# now fill it up row by row
#loc or iloc both work here since the index is natural numbers
for x in range(qb_min,qb_max):
    i = x - qb_min 
    df_pairs.loc[i] = list(optimal_split_viewer('QB',int(x/qb_adjust)))
for x in range(rb_min,rb_max):
    i = len(df_pairs)+1
    df_pairs.loc[i] = list(optimal_split_viewer('RB',x))
for x in range(wr_min,wr_max):
    i = len(df_pairs)+1
    df_pairs.loc[i] = list(optimal_split_viewer('WR',x))
for x in range(flex_min,flex_max):
    i = len(df_pairs)+1
    df_pairs.loc[i] = list(optimal_split_viewer('Flex',x)) 

    

df_pairs.set_index('Lookup', inplace=True)


In [17]:
# Create list of position budgets, calculate full state at once. 
# Loop below DOESNT use optimal_split. 
# Optimal split run on df generated by this process! 

rem_budget = starters*buyin
splits_list=[]
incr = 3

for qx in range(qb_min,min(qb_max,rem_budget),incr):
    rem_budget = starters*buyin
    splits=[]
    for rx in range(rb_min,min(rb_max,int(rem_budget-(qx+wr_min+flex_min+te_min))),incr):
        for wx in range(wr_min,min(wr_max,int(rem_budget-(rx+qx+flex_min+te_min))),incr):
            for fx in range(flex_min,min(flex_max,int(rem_budget-(rx+qx+wx+te_min))),incr):
                for tx in range(te_min,min(te_max,int(rem_budget-(rx+qx+wx+fx))),incr):
                    splits = [qx,rx,wx,fx,tx]
                    if sum(splits)>175:
                        splits_list.append(splits)
                        
                    
#### Drop anything where total < budget to save size. About half is on huge TE budgets... 

In [18]:
len(splits_list)

698108

### Now make teams! 
- Only need to lookup values from splits_list in df_pairs. No running optimizer!
- Add up points only for initial comp; will look at teams later (next section)


In [19]:
df_pairs.drop_duplicates()
df_pairs.head()
df_te = df_op.where(df_op['Pos']=='TE').dropna()

In [20]:
#### Method 3: Using Index value
####
####

import timeit
start = timeit.default_timer()


## Teams currently using qb_adjust as accelerator of costs (based on league history)
splits_list_total_pts = []

for i in range(len(splits_list)):
    team_pts = 0 
    try:
        team_pts = (df_pairs.loc[str("QB"+str(splits_list[i][0]))][3]+
                    df_pairs.loc[str("RB"+str(splits_list[i][1]))][3]+
                    df_pairs.loc[str("WR"+str(splits_list[i][2]))][3]+
                    df_pairs.loc[str("Flex"+str(splits_list[i][3]))][3]+
                    df_te.iloc[(np.searchsorted(-(df_te['Auction Value'].values), -(splits_list[i][4]))).clip(0)][-8])
    except: 
        team_pts = 0
    splits_list_total_pts.append(team_pts)
    if len(splits_list_total_pts)%10000 ==0:
        print(len(splits_list_total_pts))
    
stop = timeit.default_timer()
mper_run = (stop-start)/60


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000


In [21]:
# 1000 runs w. optimize_split = 9.7m
# 1000 runs w. .where.dropna = 5.16m
# 1000 runs w. index lookup = 0.75m
# 1000 runs w index, no TE = 0.15m
# 1000 runs w index; searchsorted TE = 0.28m
# Last 3 might be 10x faster; possibly used range of 10 instead of 1! Turns out initiating loop takes a while; execution is shorter. 
# 1000 runs w current implementation = .035m

mper_run*len(splits_list)/100/60

1236.0525142735385

In [22]:
print("Hours for split_list: ","%.1f" % (mper_run*len(splits_list)/60) )

Hours for split_list:  123605.3


In [23]:
#df_results = pd.DataFrame({'QB/RB/WR/Flex/TE':splits_list, 'Team Pts':splits_list_total_pts})
#df_results.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2018 Data\Results_newQB.csv')

## Team Evaluator
1. Prints players from optimal teams. 

In [24]:
df_results =  pd.DataFrame(np.column_stack([splits_list_total_pts,splits_list]),columns=['Team_Pts','qb','rb','wr','flex','te'])
df_results.where(df_results['Team_Pts']>1).dropna().tail()

,Team_Pts,qb,rb,wr,flex,te
698103,1555.4,86.0,92.0,2.0,2.0,1.0
698104,1563.2,86.0,92.0,2.0,2.0,4.0
698105,1563.4,86.0,92.0,2.0,5.0,1.0
698106,1564.4,86.0,92.0,5.0,2.0,1.0
698107,1565.4,86.0,95.0,2.0,2.0,1.0


In [25]:
df_results =  pd.DataFrame(np.column_stack([splits_list_total_pts,splits_list]),columns=['Team_Pts','qb','rb','wr','flex','te'])
df_results = df_results.where(df_results['flex']<df_results['rb']+df_results['wr']).dropna()
df_results = df_results.sort_values('Team_Pts',ascending=False).iloc[:1000,:]
#df_results.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2018 Data\Results.csv')                            

### Single team viewer

In [26]:

i = 310

qbs = optimal_split_viewer('QB',splits_list[i][0])
rbs = optimal_split_viewer('RB',splits_list[i][1])
wrs = optimal_split_viewer('WR',splits_list[i][2])
flexs = optimal_split_viewer('Flex',splits_list[i][3])
te = df_op.iloc[df_op['Exp_PTS'].where(np.logical_and(df_op['Pos']=='TE', df_op['Auction Value']<(splits_list[i][4]))).dropna().idxmax()]

#df_team = pd.DataFrame(np.column_stack([pl1_list, pl2_list, points_list, cost_list,pl1_costlist,pl2costlist]), 
#                           columns=['Player1', 'Player2', 'Total Pts', 'Total Cost','P1 Cost','P2 Cost'])
names=[qbs[2],qbs[3],rbs[2],rbs[3],wrs[2],wrs[3], te[1], flexs[2],flexs[3]]
costs = [qbs[6],qbs[7],rbs[6],rbs[7],wrs[6],wrs[7], str(int(te[-3])), flexs[6],flexs[7]]
points = [qbs[8],qbs[9],rbs[8],rbs[9],wrs[8],wrs[9], str(int(te[-8])), flexs[8],flexs[9]]
line_entry = [qbs[2],qbs[3],rbs[2],rbs[3],wrs[2],wrs[3], te[1], flexs[2],flexs[3],qbs[6],qbs[7],rbs[6],rbs[7],wrs[6],wrs[7], str(int(te[-3])), flexs[6],flexs[7],qbs[8],qbs[9],rbs[8],rbs[9],wrs[8],wrs[9], str(int(te[-8])), flexs[8],flexs[9]]

df_team = pd.DataFrame(np.column_stack([names,points, costs]), columns=['Player', 'Points','Costs'])
df_team

,Player,Points,Costs
0,Tyrod Taylor,221,8
1,Jameis Winston,212,5
2,Jamaal Williams,104,1
3,Duke Johnson,100,0
4,Davante Adams,162,25
5,T.Y. Hilton,151,21
6,Evan Engram,110,12
7,Todd Gurley,276,63
8,Kareem Hunt,213,40


### Run all top 1000

In [28]:
best_teams = [df_results.index.values]
lines = []
for x in range (0,1000):
    i = best_teams[0][x]

    qbs = optimal_split_viewer('QB',splits_list[i][0])
    rbs = optimal_split_viewer('RB',splits_list[i][1])
    wrs = optimal_split_viewer('WR',splits_list[i][2])
    flexs = optimal_split_viewer('Flex',splits_list[i][3])
    te = df_op.iloc[df_op['Exp_PTS'].where(np.logical_and(df_op['Pos']=='TE', df_op['Auction Value']<(splits_list[i][4]))).dropna().idxmax()]

    team_pts = df_results.loc[i][0]
    line_entry = [qbs[2],qbs[3],rbs[2],rbs[3],wrs[2],wrs[3], te[1], flexs[2],flexs[3],qbs[6],qbs[7],rbs[6],rbs[7],wrs[6],wrs[7], str(int(te[-3])), flexs[6],flexs[7],qbs[8],qbs[9],rbs[8],rbs[9],wrs[8],wrs[9], str(int(te[-8])), flexs[8],flexs[9],team_pts]
    lines.append(line_entry)
    
df_teams = pd.DataFrame(np.column_stack([lines]),columns=['qb1','qb2','rb1','rb2','wr1','wr2','te1','flx1','flx2','qb1_p','qb2_p','rb1_p','rb2_p','wr1_p','wr2_p','te1_p','flx1_p','flx2_p','qb1_c','qb2_c','rb1_c','rb2_c','wr1_c','wr2_c','te1_c','flx1_c','flx2_c','team_pts'])
df_teams

,qb1,qb2,rb1,rb2,wr1,wr2,te1,flx1,flx2,qb1_p,...,qb1_c,qb2_c,rb1_c,rb2_c,wr1_c,wr2_c,te1_c,flx1_c,flx2_c,team_pts
0,Drew Brees,Alex Smith,Le'Veon Bell,Ronald Jones II,Keenan Allen,Corey Davis,Cameron Brate,Jordan Howard,Rex Burkhead,29,...,281,263,269,122,168,104,78,172,119,1576.4
1,Drew Brees,Patrick Mahomes,Jordan Howard,Alex Collins,Antonio Brown,Tyreek Hill,Cameron Brate,Jordan Howard,Rex Burkhead,29,...,281,246,172,152,207,149,78,172,119,1576.4
2,Drew Brees,Alex Smith,Alex Collins,Ronald Jones II,Antonio Brown,Tyreek Hill,Cameron Brate,Jordan Howard,Alex Collins,29,...,281,263,152,122,207,149,78,172,152,1576.4
3,Drew Brees,Alex Smith,Jordan Howard,Alex Collins,Antonio Brown,Tyreek Hill,Cameron Brate,Alex Collins,Ronald Jones II,29,...,281,263,172,152,207,149,78,152,122,1576.4
4,Drew Brees,Tyrod Taylor,Le'Veon Bell,Jordan Howard,Tyreek Hill,Jarvis Landry,Cameron Brate,Jordan Howard,Rex Burkhead,29,...,281,221,269,172,149,115,78,172,119,1576.4
5,Drew Brees,Patrick Mahomes,Jordan Howard,Rex Burkhead,Antonio Brown,Tyreek Hill,Cameron Brate,Jordan Howard,Alex Collins,29,...,281,246,172,119,207,149,78,172,152,1576.4
6,Drew Brees,Alex Smith,Le'Veon Bell,Ronald Jones II,Antonio Brown,Jarvis Landry,Cameron Brate,Ronald Jones II,Rex Burkhead,29,...,281,263,269,122,207,115,78,122,119,1576.4
7,Drew Brees,Ryan Tannehill,Le'Veon Bell,Ronald Jones II,Antonio Brown,Tyreek Hill,Cameron Brate,Ronald Jones II,Rex Burkhead,29,...,281,229,269,122,207,149,78,122,119,1576.4
8,Drew Brees,Ryan Tannehill,Ronald Jones II,Rex Burkhead,Antonio Brown,Tyreek Hill,Cameron Brate,Le'Veon Bell,Ronald Jones II,29,...,281,229,122,119,207,149,78,269,122,1576.4
9,Drew Brees,Patrick Mahomes,Ronald Jones II,Rex Burkhead,Antonio Brown,Tyreek Hill,Cameron Brate,Ezekiel Elliott,Ronald Jones II,29,...,281,246,122,119,207,149,78,252,122,1576.4


In [29]:
    
df_teams = pd.DataFrame(np.column_stack([lines]),columns=['qb1','qb2','rb1','rb2','wr1','wr2','te1','flx1','flx2','qb1_p','qb2_p','rb1_p','rb2_p','wr1_p','wr2_p','te1_p','flx1_p','flx2_p','qb1_c','qb2_c','rb1_c','rb2_c','wr1_c','wr2_c','te1_c','flx1_c','flx2_c','team_pts'])
df_teams.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2018 Data\Results_newQBs.csv')                            